In [1]:
# Load libs:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from glob import glob
from os.path import dirname, abspath, join, exists

from sklearn import linear_model

from amplify.DataGeneration import DataGenerator
# ^^^ For this to work you need to run `pip install -r requirements.txt`

In [2]:
# DataGenerator - ClearML
data_generator = DataGenerator()
# LoadData - ClearML
load_success, building_data, weather_data = data_generator.LoadData()

# # DataGenerator - Local
# data_generator = DataGenerator(use_local_data=True)
# # LoadData - Local
# load_success, building_data, weather_data = data_generator.LoadData(
#     building_data_path="../data/2021-10-19_2022-01-09_CF2.csv",
#     weather_data_path="../data/CF2_Weather_2020-2022.csv",
# )



Info: Successfully loaded building data!


/Users/christianwelling/Fourthbrain/Fourthbrain/fourthbrain_venv/lib/python3.8/site-packages/pysolar/radiation.py:50: RuntimeWarning: overflow encountered in exp
  return flux * math.exp(-1 * optical_depth * air_mass_ratio) * is_daytime
/Users/christianwelling/Fourthbrain/Fourthbrain/fourthbrain_venv/lib/python3.8/site-packages/pysolar/radiation.py:50: RuntimeWarning: invalid value encountered in multiply
  return flux * math.exp(-1 * optical_depth * air_mass_ratio) * is_daytime


Info: Successfully loaded weather data!


In [3]:
building_data.head()

,solar,usage,solar,usage,solar,usage,solar,usage,solar,usage,solar,usage,solar,usage,solar,usage,solar,usage
,True Power (kW),True Power (kW),Total Energy (kWh),Total Energy (kWh),Reactive Energy (kVARh),Reactive Energy (kVARh),Reactive Power (kVAR),Reactive Power (kVAR),Apparent Power (kVA),Apparent Power (kVA),Apparent Energy (kVAh),Apparent Energy (kVAh),dPF,dPF,aPF,aPF,Current (A),Current (A)
Timestamp,,,,,,,,,,,,,,,,,,
2022-01-10 05:00:00+00:00,NaN,12.8,358396.1,-613740.4,1961.4,68348.8,NaN,-2.9,-0.2,13.8,236341.4,-445647.3,0.1,-0.9,0.1,-0.8,2.5,111.6
2022-01-10 04:00:00+00:00,NaN,16.3,358396.2,-613753.4,1961.5,68351.8,NaN,-2.9,-0.3,17.4,236341.7,-445662.2,0.1,-0.4,0.1,-0.4,2.7,140.1
2022-01-10 03:00:00+00:00,NaN,24.7,358396.3,-613770.1,1961.6,68352.3,NaN,1.3,-0.3,25.4,236342.1,-445684.2,0.1,0.3,0.1,0.3,2.8,205.0
2022-01-10 02:00:00+00:00,NaN,19.1,358396.4,-613794.8,1961.7,68353.3,NaN,-2.8,-0.3,20.0,236342.4,-445708.5,0.1,-0.5,0.1,-0.5,2.9,161.4
2022-01-10 01:00:00+00:00,NaN,26.2,358396.5,-613814.0,1961.8,68354.6,NaN,-2.8,-0.3,27.0,236342.8,-445729.9,0.1,-0.5,0.1,-0.5,3.0,218.2


In [4]:
weather_data.head()

,temp,pressure,humidity,clouds_all,irradiance,irridance
index,,,,,,
2021-10-19 04:00:00+00:00,10.07,1017.0,66.0,20.0,0.0,0.0
2021-10-19 05:00:00+00:00,9.51,1017.0,70.0,20.0,0.0,0.0
2021-10-19 06:00:00+00:00,9.28,1017.0,71.0,20.0,0.0,0.0
2021-10-19 07:00:00+00:00,9.12,1017.0,72.0,20.0,0.0,0.0
2021-10-19 08:00:00+00:00,9.12,1017.0,73.0,20.0,0.0,0.0


In [5]:
# What is the shape of the data?
print(f"Building Data Shape: {building_data.shape}")
print(f"Weather Data Shape: {weather_data.shape}")

Building Data Shape: (1993, 18)
Weather Data Shape: (1994, 6)


In [6]:
# Extract just the power generation data from building_data
power_gen_data = building_data.usage
power_gen_data.head()

,True Power (kW),Total Energy (kWh),Reactive Energy (kVARh),Reactive Power (kVAR),Apparent Power (kVA),Apparent Energy (kVAh),dPF,aPF,Current (A)
Timestamp,,,,,,,,,
2022-01-10 05:00:00+00:00,12.8,-613740.4,68348.8,-2.9,13.8,-445647.3,-0.9,-0.8,111.6
2022-01-10 04:00:00+00:00,16.3,-613753.4,68351.8,-2.9,17.4,-445662.2,-0.4,-0.4,140.1
2022-01-10 03:00:00+00:00,24.7,-613770.1,68352.3,1.3,25.4,-445684.2,0.3,0.3,205.0
2022-01-10 02:00:00+00:00,19.1,-613794.8,68353.3,-2.8,20.0,-445708.5,-0.5,-0.5,161.4
2022-01-10 01:00:00+00:00,26.2,-613814.0,68354.6,-2.8,27.0,-445729.9,-0.5,-0.5,218.2


In [7]:
# Sort the data
power_gen_data.sort_values('Timestamp')

,True Power (kW),Total Energy (kWh),Reactive Energy (kVARh),Reactive Power (kVAR),Apparent Power (kVA),Apparent Energy (kVAh),dPF,aPF,Current (A)
Timestamp,,,,,,,,,
2021-10-19 04:00:00+00:00,11.3,-647338.9,71915.0,-3.1,12.4,-471100.5,-0.9,-0.9,100.2
2021-10-19 05:00:00+00:00,13.2,-647325.3,71911.8,-3.1,14.2,-471087.5,-0.9,-0.9,115.1
2021-10-19 06:00:00+00:00,12.7,-647312.1,71908.7,-3.0,13.7,-471072.6,-0.9,-0.9,111.4
2021-10-19 07:00:00+00:00,17.2,-647295.7,71905.6,-2.9,18.1,-471057.6,-0.9,-0.9,147.3
2021-10-19 08:00:00+00:00,17.0,-647278.7,71902.7,-2.9,17.8,-471039.2,-0.9,-0.9,144.9
...,...,...,...,...,...,...,...,...,...
2022-01-10 01:00:00+00:00,26.2,-613814.0,68354.6,-2.8,27.0,-445729.9,-0.5,-0.5,218.2
2022-01-10 02:00:00+00:00,19.1,-613794.8,68353.3,-2.8,20.0,-445708.5,-0.5,-0.5,161.4
2022-01-10 03:00:00+00:00,24.7,-613770.1,68352.3,1.3,25.4,-445684.2,0.3,0.3,205.0


In [8]:
# extract just the True Power
y_data = power_gen_data['True Power (kW)']
y_data.head()

Timestamp
2022-01-10 05:00:00+00:00    12.8
2022-01-10 04:00:00+00:00    16.3
2022-01-10 03:00:00+00:00    24.7
2022-01-10 02:00:00+00:00    19.1
2022-01-10 01:00:00+00:00    26.2
Name: True Power (kW), dtype: float64

In [9]:
weather_data.index[0]

Timestamp('2021-10-19 04:00:00+0000', tz='UTC')

In [38]:
# format data for training...
# [1,2,....] <= was a column
# do we do (n, 1, n timesteps, n features per timestep) (3,4,5 => 2 days)?

def CreateXY(x_data, y_data, date_list, lag=48):
    x_out = []
    y_out = []

    for i_date in range(len(date_list)):
        # create new hour object
        lag_date = date_list[i_date] - dt.timedelta(hours=lag)
        predict_date = date_list[i_date] + dt.timedelta(hours=lag)

        # check lag does not preceded earliest data point
        if lag_date < x_data.index[0]: 
            lag_date = x_data.index[0]

        # check prediction date is not past date we have data for
        if predict_date > x_data.index[-1]:
            predict_date = x_data.index[-1]

        # TODO: remove the actual date from the feature list?
        x_out.append(x_data[str(lag_date):str(date_list[i_date])].values)
        y_out.append(y_data[str(predict_date)].values)

    return x_out, y_out

In [11]:
weather_data.index[-1]

Timestamp('2022-01-10 05:00:00+0000', tz='UTC')

In [12]:
import random
# Create random list of dates
#TODO: note, below isn't quite right b/c the end_date isn't at the 00:00:00 mark for the day but at something like 5:00)
start_date = weather_data.index[0]
end_date = weather_data.index[-1]
num_days = int(weather_data.shape[0]/24)
date_list = [end_date - dt.timedelta(days=x) for x in range(num_days)]

test_train_split = 0.7
random_date_list = random.sample(date_list, int(test_train_split*len(date_list)))

In [34]:
temp_x = weather_data[str(start_date):str(end_date)]
temp_x

,temp,pressure,humidity,clouds_all,irradiance,irridance
index,,,,,,
2021-10-19 04:00:00+00:00,10.07,1017.0,66.0,20.0,0.0,0.0
2021-10-19 05:00:00+00:00,9.51,1017.0,70.0,20.0,0.0,0.0
2021-10-19 06:00:00+00:00,9.28,1017.0,71.0,20.0,0.0,0.0
2021-10-19 07:00:00+00:00,9.12,1017.0,72.0,20.0,0.0,0.0
2021-10-19 08:00:00+00:00,9.12,1017.0,73.0,20.0,0.0,0.0
...,...,...,...,...,...,...
2022-01-10 01:00:00+00:00,4.59,1016.0,90.0,90.0,0.0,0.0
2022-01-10 02:00:00+00:00,4.59,1016.0,90.0,90.0,0.0,0.0
2022-01-10 03:00:00+00:00,4.59,1016.0,90.0,90.0,0.0,0.0


In [36]:
[temp_x.values]

[array([[  10.07, 1017.  ,   66.  ,   20.  ,    0.  ,    0.  ],
        [   9.51, 1017.  ,   70.  ,   20.  ,    0.  ,    0.  ],
        [   9.28, 1017.  ,   71.  ,   20.  ,    0.  ,    0.  ],
        ...,
        [   4.59, 1016.  ,   90.  ,   90.  ,    0.  ,    0.  ],
        [   4.59, 1016.  ,   90.  ,   90.  ,    0.  ,    0.  ],
        [   4.59, 1016.  ,   90.  ,   90.  ,    0.  ,    0.  ]])]

In [16]:
y_data[str(start_date)].values

array([11.3])

In [39]:
x_train, y_train = CreateXY(weather_data, y_data, random_date_list, 48)

In [42]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [43]:
x_train.shape

(58, 49, 6)

In [44]:
y_train.shape

(58, 1)

In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from keras import Input
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.layers import Bidirectional
from keras.layers import TimeDistributed

In [46]:
def CreateModel(n_layer = 1, n_lag = 48, n_ft = 10, n_outputs=1):
    lstm_input = Input(shape=(n_lag + 1 , n_ft))
    lstm_layer = LSTM(n_layer, activation='relu')(lstm_input)
    x = Dense(n_outputs)(lstm_layer)
    model = Model(inputs=lstm_input, outputs=x)
    return model

In [49]:
n_layer = 6
n_lag = 48
n_ft = 6
n_outputs = 1
model = CreateModel(n_layer, n_lag, n_ft, n_outputs)

In [55]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 49, 6)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 6)                 312       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 319
Trainable params: 319
Non-trainable params: 0
_________________________________________________________________


In [56]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
#model.compile(loss=keras.losses.MeanAbsoluteError(), optimizer=optimizer)
model.compile(tf.optimizers.Adam(learning_rate=0.0003),
      loss='binary_crossentropy',
      metrics=['acc'])

In [57]:
history = model.fit(x_train, y_train, epochs=100, batch_size=1, validation_split=0.2, shuffle=False)

Epoch 1/100
46/46 [==============================] - 4s 28ms/step - loss: -208.0858 - acc: 0.0000e+00 - val_loss: -216.6663 - val_acc: 0.0000e+00
Epoch 2/100
46/46 [==============================] - 1s 20ms/step - loss: -208.0858 - acc: 0.0000e+00 - val_loss: -216.6663 - val_acc: 0.0000e+00
Epoch 3/100
46/46 [==============================] - 1s 19ms/step - loss: -208.0858 - acc: 0.0000e+00 - val_loss: -216.6663 - val_acc: 0.0000e+00
Epoch 4/100
46/46 [==============================] - 1s 18ms/step - loss: -208.0858 - acc: 0.0000e+00 - val_loss: -216.6663 - val_acc: 0.0000e+00
Epoch 5/100
46/46 [==============================] - 1s 23ms/step - loss: -208.0858 - acc: 0.0000e+00 - val_loss: -216.6663 - val_acc: 0.0000e+00
Epoch 6/100
46/46 [==============================] - 1s 21ms/step - loss: -208.0858 - acc: 0.0000e+00 - val_loss: -216.6663 - val_acc: 0.0000e+00
Epoch 7/100
46/46 [==============================] - 1s 21ms/step - loss: -208.0858 - acc: 0.0000e+00 - val_loss: -216.6663 

KeyboardInterrupt: 

In [52]:
# These results seems a little bogus, but I will play around with it 